In [9]:
import pandas as pd
import joblib
import numpy as np
from xgboost import XGBRegressor
from category_encoders import TargetEncoder
import sklearn

In [2]:
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
sample = pd.read_csv('sample_submission.csv')
test_set = pd.read_csv('test_set.csv')
df1 = pd.read_csv('dataset_1.csv')
df2 = pd.read_csv('dataset_2.csv')
df3 = pd.read_csv('dataset_3.csv')
df4 = pd.read_csv('dataset_4.csv')
df5 = pd.read_csv('dataset_5.csv')
df6 = pd.read_csv('dataset_6.csv')
df1000_1 = pd.read_csv('dataset_1000_1.csv')
df1000_2 = pd.read_csv('dataset_1000_2.csv')
df1000_3 = pd.read_csv('dataset_1000_3.csv')
dfv2_1 = pd.read_csv('dataset_v2_1.csv')
dfv2_2 = pd.read_csv('dataset_v2_2.csv')
df_yanaser_1 = pd.read_csv('yanaser_1.csv')
df_yanaser_2 = pd.read_csv('yanaser_2.csv')
df_ma_1=pd.read_csv('dataset_MA_200_random.csv')
df_ma_2=pd.read_csv('dataset_MA_200_random_2.csv')
df_ma_3=pd.read_csv('dataset_MA_ordonne_1.csv')
df_ma_4=pd.read_csv('dataset_MA_ordonne_2.csv')
df_ma_5=pd.read_csv('dataset_MA_ordonne_3.csv')

In [3]:
df = pd.concat([df1,df2,df3,df4,df5,df6,df1000_1,df1000_2,df1000_3,dfv2_1,dfv2_2
                ,df_yanaser_1,df_yanaser_2,df_ma_1,df_ma_2,df_ma_3,df_ma_4,df_ma_5])

In [4]:
df_final = df.join(hotels, on=['hotel_id', 'city'])
dataset = df_final.copy()
dataset = dataset.drop(['Unnamed: 0','hotel_id','avatar_id'], axis=1)
test_set_joined = test_set.join(hotels, on=['hotel_id', 'city'])
index = test_set_joined['index']
y = df_final['price']
X = df_final.drop(['Unnamed: 0','price'], axis=1)
X_test = test_set_joined[['hotel_id','stock','city','date','language','mobile', 'avatar_id','group','brand','parking','pool','children_policy']]
X_ids = X[['hotel_id','avatar_id']]
X_test_ids = X_test[['hotel_id','avatar_id']]
X = X.drop(['hotel_id','avatar_id'], axis=1)
X_test = X_test.drop(['hotel_id','avatar_id'], axis=1)

In [5]:
from category_encoders import TargetEncoder
df_encoder = df_final.drop(['Unnamed: 0','hotel_id','avatar_id'], axis=1)
encoder1 = TargetEncoder()
encoder2 = TargetEncoder()
encoder3 = TargetEncoder()
encoder4 = TargetEncoder()
encoder1.fit(df_encoder['city'],df_encoder['price'])
encoder2.fit(df_encoder['language'],df_encoder['price'])
encoder3.fit(df_encoder['group'],df_encoder['price'])
encoder4.fit(df_encoder['brand'],df_encoder['price'])
df_encoder['city_encoded'] = encoder1.transform(df_encoder['city'])
df_encoder['language_encoded'] = encoder2.transform(df_encoder['language'])
df_encoder['group_encoded'] = encoder3.transform(df_encoder['group'])
df_encoder['brand_encoded'] = encoder4.transform(df_encoder['brand'])

XX_test = X_test.copy()
XX_test['city_encoded']=encoder1.transform(XX_test['city'])
XX_test['language_encoded']=encoder2.transform(XX_test['language'])
XX_test['group_encoded']=encoder3.transform(XX_test['group'])
XX_test['brand_encoded']=encoder4.transform(XX_test['brand'])

y = df_encoder.iloc[:, 0].values.reshape(-1,1)
df_train = df_encoder.drop(['price','city','language','group','brand'], axis=1)
XX_test = XX_test.drop(['city','language','group','brand'], axis=1)

c:\Users\lahlo\AppData\Local\Programs\Python\Python310\lib\site-packages\category_encoders\target_encoder.py:123: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
c:\Users\lahlo\AppData\Local\Programs\Python\Python310\lib\site-packages\category_encoders\target_encoder.py:128: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [6]:
df_train.head()

,stock,date,mobile,parking,pool,children_policy,city_encoded,language_encoded,group_encoded,brand_encoded
0,0,5,1,1,0,0,171.410442,166.531312,96.621702,96.621702
1,0,5,1,0,0,0,171.410442,166.531312,96.621702,96.621702
2,6,5,1,0,0,0,171.410442,166.531312,195.028877,207.409376
3,5,5,1,1,0,0,171.410442,166.531312,195.028877,308.324906
4,1,5,1,0,0,0,171.410442,166.531312,228.795960,154.838566


In [7]:
XX_test.head()

,stock,date,mobile,parking,pool,children_policy,city_encoded,language_encoded,group_encoded,brand_encoded
0,46,21,0,1,0,0,113.01378,156.262152,137.366493,149.459778
1,32,21,0,1,1,0,113.01378,156.262152,195.028877,207.409376
2,12,21,0,1,0,0,113.01378,156.262152,96.621702,96.621702
3,10,21,0,0,0,0,113.01378,156.262152,195.028877,80.479911
4,42,21,0,1,1,1,113.01378,156.262152,195.028877,308.324906


In [12]:
import time
from xgboost import XGBRegressor
start = time.time()
xgb_model = XGBRegressor()
xgb_model.fit(df_train,y)
end = time.time()
print(end-start)

10.367069005966187


In [13]:
search_space = {
    "n_estimators" : [100, 200, 500,1000],
    "max_depth" : [3, 6, 9],
    "gamma" : [0.01, 0.1],
    "learning_rate" : [0.001, 0.01, 0.1, 1]
}

In [14]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'positive_likelihood_ratio', 'neg_negative_likelihood_ratio', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weig

In [15]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor()
from sklearn.model_selection import GridSearchCV
# make a GridSearchCV object
GS = GridSearchCV(estimator = xgb_model,
                  param_grid = search_space,
                  scoring = 'neg_mean_squared_error', #sklearn.metrics.SCORERS.keys()
                  refit = "r2",
                  cv = 2,
                  verbose = 4)

In [16]:
GS.fit(df_train,y)

Fitting 2 folds for each of 96 candidates, totalling 192 fits
[CV 1/2] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=100;, score=-29163.588 total time=   2.7s
[CV 2/2] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=100;, score=-27089.382 total time=   2.0s
[CV 1/2] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=200;, score=-24270.582 total time=   3.7s
[CV 2/2] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=200;, score=-22281.397 total time=   3.5s
[CV 1/2] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=500;, score=-14182.656 total time=   9.6s
[CV 2/2] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=500;, score=-12571.414 total time=   8.6s
[CV 1/2] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=1000;, score=-6131.884 total time=  18.1s
[CV 2/2] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=1000;, score=-5146.297 total time=  18.1s
[CV 1/2] END gamma=0.01, l

GridSearchCV(cv=2,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'gamma': [0.01, 0.1],
                         'learning_rate': [0.001, 0.01, 0.1, 1],
                         'max_depth': [3, 6, 9],
                         'n_estimators': [100, 200, 500, 1000]},
             refit='r2', scoring='neg_mean_squared_error', verbose=4)

In [17]:
print(GS.best_estimator_) 

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.1, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=9, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)


In [18]:
print(GS.best_estimator_)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.1, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=9, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)


In [21]:
bestmodel = XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.1, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=9, max_leaves=0, min_child_weight=1,
             monotone_constraints='()', n_estimators=1000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0)


In [27]:
bestmodel = XGBRegressor(n_estimators=1000,max_depth=9,gamma=0.1,learning_rate=0.1)

In [28]:
bestmodel.fit(df_train,y)
bestmodel_prediction = bestmodel.predict(XX_test)

In [29]:
print(bestmodel_prediction)
prediction = pd.Series(bestmodel_prediction)
submission_grid = pd.concat([index,prediction], axis=1)
submission_grid.columns = ['index','price']
submission_grid.head()

[110.253716 149.16968   71.495384 ... 166.77762  191.10028  166.77762 ]


,index,price
0,0,110.253716
1,1,149.169678
2,2,71.495384
3,3,60.664608
4,4,200.756729


In [30]:
submission_grid.to_csv('gridsearchpredi_2.csv',index=False)